# From Boyer-Lindquist to Kerr coordinates in Kerr spacetime

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://relativite.obspm.fr/blackholes/).

The involved computations are based on tools developed through the [SageManifolds](https://sagemanifolds.obspm.fr) project.

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [1]:
version()

'SageMath version 9.1.beta8, Release Date: 2020-03-18'

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [2]:
%display latex

To speed up the computations, we ask for running them in parallel on 8 threads:

In [3]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the Boyer-Lindquist domain of Kerr spacetime) as a 4-dimensional Lorentzian manifold:

In [4]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


Let us declare the **Boyer-Lindquist coordinates** via the method `chart()`, the argument of which is a string expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [5]:
BL.<t,r,th,ph> = M.chart(r't r th:(0,pi):\theta ph:(0,2*pi):periodic:\phi') 
print(BL) ; BL

Chart (M, (t, r, th, ph))


Chart (M, (t, r, th, ph))

In [6]:
BL[0], BL[1]

(t, r)

In [7]:
BL.coord_range()

t: (-oo, +oo); r: (-oo, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

<h2>Metric tensor</h2>

<p>The 2 parameters $m$ and $a$ of the Kerr spacetime are declared as symbolic variables:</p>

In [8]:
var('m, a', domain='real')

(m, a)

In [9]:
assume(m>a)
assume(m^2 - a^2 >0)

We get the (yet undefined) spacetime metric by 

In [10]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [11]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [12]:
g[:]

[                                  2*m*r/(a^2*cos(th)^2 + r^2) - 1                                                                 0                                                                 0                          -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                                                0                         (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2)                                                                 0                                                                 0]
[                                                                0                                                                 0                                               a^2*cos(th)^2 + r^2                                                                 0]
[                         -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)                                                                 0                                                                 0 (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2]

<p>The list of the non-vanishing components:</p>

In [13]:
g.display_comp()

g_t,t = 2*m*r/(a^2*cos(th)^2 + r^2) - 1 
g_t,ph = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_r,r = (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) 
g_th,th = a^2*cos(th)^2 + r^2 
g_ph,t = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_ph,ph = (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2

<h2>Levi-Civita Connection</h2>

<p>The Levi-Civita connection $\nabla$ associated with $g$:</p>

In [14]:
nabla = g.connection() ; print(nabla)

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional Lorentzian manifold M


<p>Let us verify that the covariant derivative of $g$ with respect to $\nabla$ vanishes identically:</p>

In [15]:
nabla(g).display()

nabla_g(g) = 0

# 3+1 Kerr coordinates

In [16]:
KC.<ti, r, th, tph> = M.chart(r'ti:\tilde{t} r th:(0,pi):\theta tph:(0,2*pi):periodic:\tilde{\phi}')
KC

Chart (M, (ti, r, th, tph))

The transition from Boyer-Lindquist to 3+1 Kerr coordinates:

In [17]:
sma = sqrt(m^2 - a^2)
rp = m + sma
rm = m - sma
BL_to_KC = BL.transition_map(KC, [t + m/sma*(rp*ln((r-rp)/(2*m)) - rm*ln((r-rm)/(2*m))),
                                  r,
                                  th,
                                  ph + a/(2*sma)*ln((r-rp)/(r-rm))])
BL_to_KC.display()

ti = ((m + sqrt(-a^2 + m^2))*log(-1/2*(m - r + sqrt(-a^2 + m^2))/m) - (m - sqrt(-a^2 + m^2))*log(-1/2*(m - r - sqrt(-a^2 + m^2))/m))*m/sqrt(-a^2 + m^2) + t
r = r
th = th
tph = 1/2*a*log((m - r + sqrt(-a^2 + m^2))/(m - r - sqrt(-a^2 + m^2)))/sqrt(-a^2 + m^2) + ph

In [18]:
BL_to_KC.inverse().display()

t = (2*a^2*m*log(2) - 2*m^3*log(2) - sqrt(a + m)*sqrt(-a + m)*m^2*log(-(sqrt(a + m)*sqrt(-a + m) - m + r)/(sqrt(a + m)*sqrt(-a + m) + m - r)) + (a^2 - m^2)*ti - (a^2*m - m^3)*log(a^2 - 2*m*r + r^2) + 2*(a^2*m - m^3)*log(m))/(a^2 - m^2)
r = r
th = th
ph = -1/2*(sqrt(a + m)*a*sqrt(-a + m)*log(-(a^2 - 2*m*r + r^2)/(a^2 - 2*sqrt(a + m)*sqrt(-a + m)*(m - r) - 2*m^2 + 2*m*r - r^2)) - 2*(a^2 - m^2)*tph)/(a^2 - m^2)

In [19]:
for v in KC.frame():
    show(v.display())

d/dti = d/dt

d/dr = -2*m*r/(a^2 - 2*m*r + r^2) d/dt + d/dr - a/(a^2 - 2*m*r + r^2) d/dph

d/dth = d/dth

d/dtph = d/dph

### Components of the metric tensor w.r.t. 3+1 Kerr coordinates

In [20]:
g.display_comp(KC.frame(), KC, only_nonredundant=True)

g_ti,ti = -(a^2*cos(th)^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) 
g_ti,r = 2*m*r/(a^2*cos(th)^2 + r^2) 
g_ti,tph = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_r,r = (a^2*cos(th)^2 + 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) 
g_r,tph = -(2*a^3*m*r*sin(th)^4 + (a*r^4 + (a^3 - 4*a*m^2)*r^2 + (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) 
g_th,th = a^2*cos(th)^2 + r^2 
g_tph,tph = (2*a^2*m*r*sin(th)^4 + (a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2)

$g_{r\tilde{\phi}}$ can simplified further:

In [21]:
g13 = g[KC.frame(), 1, 3, KC]
g13

-(2*a^3*m*r*sin(th)^4 + (a*r^4 + (a^3 - 4*a*m^2)*r^2 + (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)

In [22]:
g13.factor()

-(a^4*cos(th)^2 + a^2*r^2*cos(th)^2 + 2*a^2*m*r*sin(th)^2 + a^2*r^2 - 4*m^2*r^2 + r^4)*a*sin(th)^2/((a^2*cos(th)^2 + r^2)*(a^2 - 2*m*r + r^2))

In [23]:
g13 == - a*(1 + 2*m*r/rho2)*sin(th)^2

True

We then set (using `add_comp()` to keep the components with respect to BL coordinates):

In [24]:
g.add_comp(KC.frame())[1, 3, KC] =  - a*(1 + 2*m*r/rho2)*sin(th)^2

Similarly $g_{\tilde{\phi}\tilde{\phi}}$ can simplified:

In [25]:
g33 = g[KC.frame(), 3, 3, KC]
g33

(2*a^2*m*r*sin(th)^4 + (a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2)

In [26]:
g33.factor()

(a^4*cos(th)^2 + a^2*r^2*cos(th)^2 + 2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4)*sin(th)^2/(a^2*cos(th)^2 + r^2)

In [27]:
g33 == (r^2 + a^2 + 2*a^2*m*r*sin(th)^2/rho2)*sin(th)^2

True

We then set (using `add_comp()` to keep the components with respect to BL coordinates):

In [28]:
g.add_comp(KC.frame())[3, 3, KC] = (r^2 + a^2 + 2*a^2*m*r*sin(th)^2/rho2)*sin(th)^2

In [29]:
g.display_comp(KC.frame(), KC, only_nonredundant=True)

g_ti,ti = -(a^2*cos(th)^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) 
g_ti,r = 2*m*r/(a^2*cos(th)^2 + r^2) 
g_ti,tph = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_r,r = (a^2*cos(th)^2 + 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) 
g_r,tph = -a*(2*m*r/(a^2*cos(th)^2 + r^2) + 1)*sin(th)^2 
g_th,th = a^2*cos(th)^2 + r^2 
g_tph,tph = (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2

## Principal null vectors

Let $k$ be the null vector tangent to the ingoing principal null geodesics associated
with their affine parameter $-r$; the expression of $k$ in term of the 3+1 ingoing Kerr coordinates $(\tilde{t}, r, \theta, \tilde\phi)$ is 
$$k = \frac{\partial}{\partial\tilde{t}} - \frac{\partial}{\partial\tilde{r}} $$
The expression of $k$ in terms of the Boyer-Lindquist coordinates is

In [30]:
k = M.vector_field(name='k')
k[KC.frame(),:,KC] = [1, -1, 0, 0]
k.display(KC)

k = d/dti - d/dr

In [31]:
k.display(BL)

k = (a^2 + r^2)/(a^2 - 2*m*r + r^2) d/dt - d/dr + a/(a^2 - 2*m*r + r^2) d/dph

Regarding the null vector tangent to the outgoing principal null geodesics, we select
one associated with a (non-affine) parameter $\lambda$ that is regular accross the two Killing horizons:

In [32]:
el = M.vector_field(name='el', latex_name=r'\ell')
el[KC.frame(),:,KC] = [1/2 + m*r/(r^2+a^2), 
                       1/2 - m*r/(r^2+a^2),
                       0,
                       a/(r^2 + a^2)]
el.display(KC)

el = (m*r/(a^2 + r^2) + 1/2) d/dti + (-m*r/(a^2 + r^2) + 1/2) d/dr + a/(a^2 + r^2) d/dtph

In [33]:
el.display(BL)

el = 1/2 d/dt + 1/2*(a^2 - 2*m*r + r^2)/(a^2 + r^2) d/dr + 1/2*a/(a^2 + r^2) d/dph

Let us check that $k$ and $\ell$ are null vectors:

In [34]:
g(k, k).expr()

0

In [35]:
g(el, el).expr()

0

Their scalar product is $-\rho^2/(r^2 + a^2)$:

In [36]:
g(k, el).expr()

-(a^2*cos(th)^2 + r^2)/(a^2 + r^2)

In [37]:
uk = k.down(g)
uk.display(BL)

-dt - (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr + (2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) dph

In [38]:
uk.display(KC)

-dti - dr + (2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) dtph

In [39]:
uk[3]

(2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)

In [40]:
uk[3].factor()

(a^4*cos(th)^2 + a^2*r^2*cos(th)^2 + 2*a^2*m*r*sin(th)^2 - 2*a^2*m*r + a^2*r^2 - 2*m*r^3 + r^4)*a*sin(th)^2/((a^2*cos(th)^2 + r^2)*(a^2 - 2*m*r + r^2))

In [41]:
uk[3] == a*sin(th)^2

True